In [1]:
"""
#TensorFlow 2.10 was the last TensorFlow release that supported GPU on native-Windows.
# !pip install "tensorflow<2.11" 
!pip install tensorflow==2.3
!pip install --upgrade pip
!pip install pandas
!pip install nltk
!pip install rake_nltk 
!pip install -U scikit-learn
!pip install transformers
!pip install matplotlib
!pip install numpy
!pip install seaborn
# For error : AttributeError: module 'keras.engine.data_adapter' has no attribute 'expand_1d'
!pip install --upgrade git+https://github.com/huggingface/transformers.git"""

'\n#TensorFlow 2.10 was the last TensorFlow release that supported GPU on native-Windows.\n# !pip install "tensorflow<2.11" \n!pip install tensorflow==2.3\n!pip install --upgrade pip\n!pip install pandas\n!pip install nltk\n!pip install rake_nltk \n!pip install -U scikit-learn\n!pip install transformers\n!pip install matplotlib\n!pip install numpy\n!pip install seaborn\n# For error : AttributeError: module \'keras.engine.data_adapter\' has no attribute \'expand_1d\'\n!pip install --upgrade git+https://github.com/huggingface/transformers.git'

In [10]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

def clean_text(text):
  text = text.lower()
  text = re.sub("[^a-zA-Z\'\-éòóôëè]", " ", text) 
  return " ".join(word_tokenize(text)[:])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Theranet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [92]:
import pandas as pd
df_model=pd.read_csv('../csv/NL_document_full_targets.csv',sep=';')
df_model=pd.read_csv('../csv/NL_document_targets.csv',sep=';')

df_model=df_model[:200]
X = df_model["text"].tolist()

y_1 = pd.get_dummies(df_model['1'])
y_2 = pd.get_dummies(df_model['2'])
y_3 = pd.get_dummies(df_model['3'])
y_4 = pd.get_dummies(df_model['4'])
y_5 = pd.get_dummies(df_model['5'])
y_6 = pd.get_dummies(df_model['6'])
y_7 = pd.get_dummies(df_model['7'])
y_8 = pd.get_dummies(df_model['8'])
y_9 = pd.get_dummies(df_model['9'])
y_10 = pd.get_dummies(df_model['10'])
y=pd.concat([y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10],axis=1,join='inner')
mapping = {i: name for i, name in enumerate(y.columns)}

#print (mapping)

print (len(y))

from sklearn.model_selection import train_test_split
# Split Train and Validation data

#print (len(X))
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#print ('X_train_val',len(X_train_val))
#print ('X_test',len(X_test))

print (len(X))
# Keep some data for inference (testing)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)
print ('X_train',len(X_train))
print ('X_test',len(X_test))
print ('X_val',len(X_val))

200
200
X_train 140
X_test 40
X_val 60


In [40]:
y.head()
y.to_csv('columns.csv',sep=';')

In [93]:
print (len(y.columns))
still=y.columns.unique() 

y = y.groupby(level=0,axis=1).sum()
print (y['excl'])

y_unique = y.loc[:,~y.columns.duplicated()].copy()
print (len(y_unique.columns))


print (y_unique['excl'])
print (len(y_unique.columns))

drop_columns=['excl','e']
y_unique.drop(['excl','e'], axis=1, inplace=True)

print (len(y_unique.columns))


266
0      0
1      1
2      0
3      0
4      1
      ..
195    1
196    0
197    1
198    0
199    0
Name: excl, Length: 200, dtype: uint8
111
0      0
1      1
2      0
3      0
4      1
      ..
195    1
196    0
197    1
198    0
199    0
Name: excl, Length: 200, dtype: uint8
111
109


In [3]:
import tensorflow as tf
tf.__version__

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9095566726646615639
xla_global_id: -1
]


In [4]:

from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
model = TFRobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels=len(set(y)))

c:\Users\Theranet\becode\GCloud\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

from transformers import TFDistilBertForSequenceClassification  
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(y)))

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_57', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [6]:
train_encodings = tokenizer(X_train, max_length=12, truncation=True, padding=True)
val_encodings = tokenizer(X_val, max_length=12, truncation=True, padding=True)
test_encodings = tokenizer(X_test, max_length=12, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))


In [7]:
OPTIMIZER =  tf.keras.optimizers.Adam(learning_rate=1)
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#METRICS = ['accuracy']

#model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
model.compile(optimizer=OPTIMIZER, loss=LOSS)
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  28453     
                                                                 
 dropout_57 (Dropout)        multiple                  0         
                                                                 
Total params: 66,981,925
Trainable params: 66,981,925
Non-trainable params: 0
_________________________________________________________________


In [8]:
tf.config.run_functions_eagerly(True)

BATCH_SIZE = 12
EPOCHS =2
history=model.fit(
    train_dataset.batch(BATCH_SIZE) ,
    epochs=EPOCHS,
    validation_data=val_dataset.batch(BATCH_SIZE)
)

Epoch 1/2
12/12 [==============================] - 19s 2s/step - loss: 9178.0498 - val_loss: 1823.8971
Epoch 2/2
12/12 [==============================] - 18s 2s/step - loss: 681.2620 - val_loss: 9.4272


In [12]:
import re

with open ('../processed_data/NL/NL_312-2019-012471.txt') as f:
    text=f.read() 
text = clean_text(text)
#print (text)

#not enough values to unpack (expected 2, got 1)
#input_ids = input_ids.unsqueeze(0)
#attention_mask = attention_mask.unsqueeze(0)

# remove max_length

encodings = tokenizer([text],truncation=True, padding=True)
ds = tf.data.Dataset.from_tensor_slices(dict(encodings))
predictions = model.predict(ds)

import numpy as np
print(mapping[np.argmax(predictions[0])])

1/1 [==============================] - 1s 1s/step
werkloosheid (andere dan economische werkloosheid voor bediendenopzegging/ontslag - organisatie en procedure
